In [1]:
%load_ext lab_black

In [2]:
from enum import IntEnum
from pathlib import Path

import numpy as np

In [13]:
class ViewDirection(IntEnum):
    LEFT_TO_RIGHT = 0
    RIGHT_TO_LEFT = 1
    TOP_TO_BOTTOM = 2
    BOTTOM_TO_TOP = 3

    @staticmethod
    def get_all():
        return [ViewDirection(i) for i in range(4)]

    def view(self, array):
        if self.value == 0:
            return array
        if self.value == 1:
            return array[:, ::-1]
        if self.value == 2:
            return array.T
        if self.value == 3:
            return array.T[:, ::-1]

    def restore(self, array):
        if self.value == 0:
            return array
        if self.value == 1:
            return array[:, ::-1]
        if self.value == 2:
            return array.T
        if self.value == 3:
            return array.T[::-1]

In [14]:
def generate_mask(data):
    mask = np.zeros_like(data, dtype=bool)
    for ri in range(len(data)):
        biggest_tree = -1
        for ci in range(len(data[ri])):
            if data[ri, ci] > biggest_tree:
                mask[ri, ci] = True
            biggest_tree = max(data[ri, ci], biggest_tree)
            if biggest_tree == 9:
                break
    return mask

---

In [15]:
data = np.array([list(r) for r in Path("data/08.txt").read_text().split("\n")]).astype(
    int
)

In [16]:
mask_l2r = generate_mask(data)
mask_r2l = generate_mask(data[:, ::-1])[:, ::-1]
mask_t2b = generate_mask(data.T).T
mask_b2t = generate_mask(data.T[:, ::-1]).T[::-1]

(mask_l2r | mask_r2l | mask_t2b | mask_b2t).sum()

1854

In [17]:
mask, *rest = [
    side.restore(generate_mask(side.view(data))) for side in ViewDirection.get_all()
]

for other in rest:
    mask = np.logical_or(mask, other)

mask.sum()

1854

---

In [7]:
def scenic_score(A, row, col):
    height = A[row, col]

    score_top = 0
    for i in range(row - 1, -1, -1):
        score_top += 1
        if A[i, col] >= height:
            break

    score_bottom = 0
    for i in range(row + 1, A.shape[0]):
        score_bottom += 1
        if A[i, col] >= height:
            break

    score_left = 0
    for i in range(col - 1, -1, -1):
        score_left += 1
        if A[row, i] >= height:
            break

    score_right = 0
    for i in range(col + 1, A.shape[1]):
        score_right += 1
        if A[row, i] >= height:
            break

    return score_top * score_left * score_bottom * score_right

In [8]:
max_scene_score = -1
for row in range(1, data.shape[0] - 1):
    for col in range(1, data.shape[1] - 1):
        max_scene_score = max(max_scene_score, scenic_score(data, row, col))
max_scene_score

527340